# Load Data

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
#LOAD DATA in a DataFrame
bucket='hf-initial-data'
folder_name='processed'
data_key = 'HF_AWS_Final.csv'
data_location = 's3://{}/{}/{}'.format(bucket, folder_name, data_key)

df = pd.read_csv(data_location)

#Enable full print of dataframe just to see everything instead of only 10 columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (188,259,265,289,325,331,346,375,377,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Allotting Imputation Methods

In [2]:
df.dtypes

Unnamed: 0                                                int64
pre_internal_VisitAnonId                                  int64
pre_internal_PatientAnonId                                int64
pre_internal_MrCareProviderAnonId                         int64
pre_internal_MrCareProviderService                       object
pre_internal_Gender                                       int64
pre_internal_Age                                          int64
pre_internal_HealthAuthorityCode                         object
pre_internal_FacilityCode                                object
pre_internal_AdmissionDate                               object
pre_internal_DischargeDate                               object
pre_internal_MRDx                                        object
pre_internal_MR_Unit                                     object
pre_internal_FiscalPeriod                                object
pre_internal_FiscalYear                                   int64
pre_internal_LOS                        

In [3]:
df.drop(df.columns[df.isna().all()],axis=1,inplace=True)
df.isna().all()

Unnamed: 0                                              False
pre_internal_VisitAnonId                                False
pre_internal_PatientAnonId                              False
pre_internal_MrCareProviderAnonId                       False
pre_internal_MrCareProviderService                      False
pre_internal_Gender                                     False
pre_internal_Age                                        False
pre_internal_HealthAuthorityCode                        False
pre_internal_FacilityCode                               False
pre_internal_AdmissionDate                              False
pre_internal_DischargeDate                              False
pre_internal_MRDx                                       False
pre_internal_MR_Unit                                    False
pre_internal_FiscalPeriod                               False
pre_internal_FiscalYear                                 False
pre_internal_LOS                                        False
pre_inte

In [ ]:
impute_info = pd.read_excel('VOCAL-HF UBC CIC Columns Missing Data 22.02.13.xlsx',engine='openpyxl')
cols_to_drop = []
# hx_columns = []
absent_columns = []
miss_zero_columns = []
miss_mean_columns=[]
miss_knn_columns=[]

#assign column names to a particular type of imputation
for i,row in impute_info.iterrows():
    method = row['Suggested Imputation Method']
    #this try-except is because the new columns names have prefixes as opposed to the ones that were labelled by Nat for the 
    #suggested imputation techniques 
    try:
        col = 'pre_internal_' + row['Column Name']
        assert col in df
        print(col,method)
    except:
        try:
            col = 'pre_CSBC_' + row['Column Name']
            assert col in df
            print(col,method)
        except:
            continue

    if method == 'Include' or method =='Missing = 0/None' or method == 'Missing = 0':
        miss_zero_columns.append(col)
    elif method == 'Exclude':
        if df[col].isnull().all():
            cols_to_drop.append(col)
        else:
            miss_mean_columns.append(col)
#         cols_to_drop.append(col)
    elif method == 'Missing = absent':
        absent_columns.append(col)
    elif method == 'Mean':
        miss_mean_columns.append(col)
    elif method == 'Impute':
        miss_knn_columns.append(col)
    else:
        print(col,method)
#         print('Other\n\n\n',col,method,'\n\n\n')





for col in miss_zero_columns:
    df[col] = df[col].fillna(0)
for col in absent_columns:
    if df[absent_columns].dtypes[col]==np.float64:
        print('float column', col)
        df[col] = df[col].fillna(0)
    else:
        print('object column', col)
        df[col] = df[col].fillna('Absent')
print(absent_columns)
df['pre_internal_LabAdm_INR']=df['pre_internal_LabAdm_INR'].fillna(1.0)
df['pre_internal_LabAdm_HbA1C']=df['pre_internal_LabAdm_HbA1C'].fillna(6.0)
df['pre_internal_LabDisch_INR']=df['pre_internal_LabDisch_INR'].fillna(1.0)
df['pre_internal_LabDisch_HbA1C']=df['pre_internal_LabDisch_HbA1C'].fillna(6.0)
df['pre_internal_ECG_Rhythm']=df['pre_internal_ECG_Rhythm'].fillna('Sinus rhythm')
df['pre_internal_ECG_QRS_Morphology']=df['pre_internal_ECG_QRS_Morphology'].fillna('Narrow')
df['pre_internal_ECG_AV_Block']=df['pre_internal_ECG_AV_Block'].fillna('No AV block')
df['pre_internal_Imaging_MR']=df['pre_internal_Imaging_MR'].fillna(0)
df['pre_internal_FollowUpAppointment']=df['pre_internal_FollowUpAppointment'].fillna(0)
# df['ExclusionReason']

# Prepare data for training

In [4]:
bnp=[]
for i,row in df.iterrows():
    t1=row['pre_internal_LabAdm_BNP']
    t2=row['pre_internal_LabAdm_NT_proBNP']
    if not pd.isnull(t1):
        if t1[0]=='<' or t1[0]=='>':
#             print(x)
            t1=t1[1:]
        bnp.append(float(t1))
    elif not pd.isnull(t2):
        if t2[0]=='<' or t2[0]=='>':
#             print(x)
            t2=t2[1:]
        bnp.append(float(t2)/6.25)
    else:
        bnp.append(np.nan)
    print(t1,t2)
    
df['pre_internal_BNP']=bnp
miss_knn_columns.append('pre_internal_BNP')

tp=[]
for i,row in df.iterrows():
    t1=row['pre_internal_LabAdm_TroponinT']
    t2=row['pre_internal_LabAdm_TroponinI']
    if not pd.isnull(t1):
        if t1[0]=='<' or t1[0]=='>':
#             print(x)
            t1=t1[1:]
        tp.append(float(t1))
    elif not pd.isnull(t2):
        if t2[0]=='<' or t2[0]=='>':
#             print(x)
            t2=t2[1:]
        tp.append(float(t2)*1000)
    else:
        tp.append(np.nan)
#     print(t1,t2)
    
df['pre_internal_Troponin']=tp
miss_knn_columns.append('pre_internal_Troponin')

cols_to_drop.extend(['pre_internal_LabAdm_BNP','pre_internal_Death','pre_internal_LabAdm_NT_proBNP','pre_internal_LabAdm_TroponinI','pre_internal_LabAdm_TroponinT','pre_internal_DeathInHospital'])


df.drop(cols_to_drop,axis=1,inplace=True)

# df['ExclusionReason']

pre_internal_ELOS Mean
pre_internal_ALOS_ELOS Mean
pre_internal_Palliative Exclude
pre_internal_hx_CKD Include
pre_internal_hx_COPD Include
pre_internal_hx_Asthma Include
pre_internal_hx_Malignancy Include
pre_internal_hx_LiverDisease Include
pre_internal_hx_Anemia Include
pre_internal_hx_PUD Include
pre_internal_hx_Dementia Include
pre_internal_hx_MI Include
pre_internal_hx_Angina Include
pre_internal_hx_CABG Include
pre_internal_hx_PCI Include
pre_internal_hx_CVA Include
pre_internal_hx_TIA Include
pre_internal_hx_PAD Include
pre_internal_hx_DM Include
pre_internal_hx_HTN Include
pre_internal_hx_Smoking Include
pre_internal_hx_AlcoholExcess Include
pre_internal_hx_AF Include
pre_internal_hx_AFL Include
pre_internal_hx_VTE Include
pre_internal_hx_ExistingHF Include
pre_internal_hx_HF_Hospitalization Include
pre_internal_hx_HeartTransplant Include
pre_internal_Dyspnea Exclude
pre_internal_Orthopnea Exclude
pre_internal_PND Exclude
pre_internal_Swelling Exclude
pre_internal_Fatigue Excl

float column pre_internal_MedDisch_ACEI
object column pre_internal_MedDisch_ACEI_Contraindication
float column pre_internal_MedDisch_ARB
object column pre_internal_MedDisch_ARB_Contraindication
float column pre_internal_MedDisch_BB
object column pre_internal_MedDisch_BB_Contraindication
float column pre_internal_MedDisch_MRA
object column pre_internal_MedDisch_MRA_Contraindication
float column pre_internal_MedDisch_ARNI
object column pre_internal_MedDisch_ARNI_Contraindication
float column pre_internal_MedDisch_DIG
float column pre_internal_MedDisch_Hydralazine
float column pre_internal_MedDisch_NitrateOralOrPatch
float column pre_internal_MedDisch_LoopDiuretic
float column pre_internal_MedDisch_ThiazideDiuretic
float column pre_internal_MedDisch_ASA
float column pre_internal_MedDisch_OtherAntiplatelet
float column pre_internal_MedDisch_Warfarin
object column pre_internal_MedDisch_WarfarinContraindication
float column pre_internal_MedDisch_NOAC
object column pre_internal_MedDisch_NOAC_

nan 2370
nan 9044
nan 2283
nan nan
nan nan
nan 2353
nan 2810
nan 7781
nan nan
nan 7203
1583 nan
nan nan
nan 2079
nan 814
nan nan
nan 7262
nan 7471
nan nan
nan 19469
nan 1977
nan nan
nan 25102
nan 1578
1978 nan
nan 7668
5000 nan
nan nan
nan 33888
nan 4846
nan nan
nan nan
nan nan
nan nan
nan 994
2273 nan
152 nan
nan 1789
nan nan
nan 4801
nan 2297
nan 2055
nan nan
nan 25318
nan 6481
nan 7069
nan nan
nan nan
nan 6868
nan nan
nan nan
nan 1872
5000 nan
551 nan
nan 22683
nan 2704
nan 1641
nan 655
nan 3928
381 nan
943 nan
nan 4281
nan nan
1225 nan
nan nan
nan 6658
255 nan
nan 3609
nan nan
1186 nan
nan 21359
nan 18227
nan 2122
nan 2925
2647 nan
1319 nan
nan 21978
nan nan
nan nan
nan nan
nan nan
nan 770
nan 38645
nan 2673
1549 nan
nan nan
3249 nan
nan nan
nan 5612
nan 6075
nan 1474
nan nan
nan nan
nan nan
nan 4438
nan 8370
nan nan
nan 10863
207 nan
1137 nan
nan 4128
nan nan
nan nan
nan nan
nan 2285
nan nan
nan 13978
nan nan
272 nan
nan 4478
nan 8507
nan nan
nan nan
nan 6832
nan 2263
337 nan
nan 

In [5]:
import datetime


for hosp in range(1,55,1):
    adm_day=[]
    adm_month=[]
    discharge_day=[]
    discharge_month=[]
    for i,row in df.iterrows():
        date = row['post_ADMISSION_DATE_'+str(hosp)]
        if pd.isnull(date):
            adm_day.append(np.nan)
            adm_month.append(np.nan)
            discharge_day.append(np.nan)
            discharge_month.append(np.nan)
        else:
#         print(date)
            at=datetime.datetime.strptime(date,'%Y-%m-%d')
            adm_day.append(at.weekday())
            adm_month.append(at.month)

            dt=datetime.datetime.strptime(date,'%Y-%m-%d') + datetime.timedelta(days=row['post_LOS_'+str(hosp)])
            discharge_day.append(dt.weekday())
            discharge_month.append(dt.month)
    df['post_ADMISSION_DATE_DAY_'+str(hosp)]=adm_day
    df['post_ADMISSION_DATE_MONTH_'+str(hosp)]=adm_month

    df['post_Discharge_Day_'+str(hosp)]=discharge_day
    df['post_Discharge_Month_'+str(hosp)]=discharge_month

In [6]:
df['post_n_rehosp']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24       NaN
25       NaN
26       NaN
27       NaN
28       NaN
29       NaN
30       NaN
31       NaN
32       NaN
33       NaN
34      10.0
35       3.0
36       2.0
37       9.0
38      15.0
39       1.0
40       6.0
41       1.0
42       1.0
43       NaN
44       1.0
45       2.0
46       NaN
47       2.0
48       3.0
49       1.0
50       2.0
51      54.0
52       1.0
53       NaN
54       3.0
55       NaN
56       NaN
57       2.0
58       1.0
59       NaN
60       2.0
61      12.0
62       NaN
63       2.0
64       NaN
65      12.0
66       5.0
67       NaN
68       3.0
69       2.0
70       NaN
71       NaN
72       1.0
73       2.0
74       1.0
75       2.0
76       8.0

In [7]:
def cap(l,n=5.0):
    r=[]
    for x in l:
        if x>n:
            r.append(n)
        else:
            r.append(x)
    return r
# df_not_missing['HospitalizationCountCapped']=cap(df_not_missing['HospitalizationCount'])
df['post_n_rehosp_cap']=cap(df['post_n_rehosp'])




# print(list(df.columns))
t_death = []
for i,row in df.iterrows():
    if not pd.isna(row['post_Death_date']):
        hosp = int(row['post_n_rehosp'])
        adm_date = row['post_ADMISSION_DATE_'+str(hosp)]
#         print(hosp)
        dt=datetime.datetime.strptime(adm_date,'%Y-%m-%d') + datetime.timedelta(days=row['post_LOS_'+str(hosp)])
        t = datetime.datetime.strptime(row['post_Death_date'],'%m/%d/%Y') - dt
        t=t.days
    else:
        t = np.inf
    t_death.append(t)
df['post_Time_to_death']=t_death
# for d in days_to_check:
#     df['Death_'+d]=df[]
# print(df['Time_to_death'])


days_to_check=[90,120]
for d in days_to_check:
    df[('post_Death_'+str(d))]=(df['post_Time_to_death']<=d).astype(int)

# print(df['Death_30'])

t_rehosp=[]

for i,row in df.iterrows():
    if not pd.isna(row['post_n_rehosp']):
        hosp = int(row['post_n_rehosp'])
        t=row['post_LOS_'+str(hosp)]
    else:
        t=np.inf
    t_rehosp.append(t)
df['post_days_for_last_rehosp']=t_rehosp
# for d in days_to_check:
#     df['Death_'+d]=df[]
# print(df['Time_to_death'])


days_to_check=[90,120]
for d in days_to_check:
    df[('post_Hospitalization_'+str(d))]=(df['post_days_for_last_rehosp']<=d).astype(int)


# Begin Training Models

In [8]:
%pip install imbalanced-learn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.tree import export_graphviz

from copy import deepcopy

print(df.shape)
cat_columns = list(df.select_dtypes(include=['object']).columns)#select those columns which are of 'object' type
df_factorized=deepcopy(df)
df_factorized[cat_columns] = df_factorized[cat_columns].apply(lambda x: pd.factorize(x)[0])
df_factorized = df_factorized.dropna(axis=0,subset=['post_n_rehosp'])

# df_not_missing = df_not_missing.drop(['CreateDate','AdmissionDate','DischargeDate','PatientAnonId','Unnamed: 0'],axis=1)
print(df.shape)

#drop 'unnamed' columns that happen due to pandas merging
df_factorized.drop(df.filter(regex="Unname"),axis=1, inplace=True)
#drop RefLimit columns as there is not enough data for them
df_factorized.drop(df.filter(regex="RefLimit"),axis=1, inplace=True)
print(df_factorized.dtypes)

#drop some columns to be imputed since we dropped them
miss_duplicate_mean_columns=[c for c in miss_mean_columns if c in list(df_factorized.columns)]
miss_mean_columns=miss_duplicate_mean_columns

# print(df_factorized.dtypes)
# print(df_factorized['MedDisch_NSAID'])

Note: you may need to restart the kernel to use updated packages.
(1844, 897)
(1844, 897)
pre_internal_VisitAnonId                                  int64
pre_internal_PatientAnonId                                int64
pre_internal_MrCareProviderAnonId                         int64
pre_internal_MrCareProviderService                        int64
pre_internal_Gender                                       int64
pre_internal_Age                                          int64
pre_internal_HealthAuthorityCode                          int64
pre_internal_FacilityCode                                 int64
pre_internal_AdmissionDate                                int64
pre_internal_DischargeDate                                int64
pre_internal_MRDx                                         int64
pre_internal_MR_Unit                                      int64
pre_internal_FiscalPeriod                                 int64
pre_internal_FiscalYear                                   int64
pre_internal_L

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.inspection import permutation_importance
def model_train(m,parameters):
    
    y = df_factorized['post_n_rehosp_cap'].values
    x_data = df_factorized.drop(df_factorized.filter(regex="post"),axis=1)
    #x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data)).values
    x = x_data
    print(x.shape)
#     print(x.dtypes)
    print(len(df_factorized.columns[:-1]))

    outs=[]
    fis=[]
    for rs in [1,2,3]:
        skf = StratifiedKFold(n_splits=8,shuffle=True,random_state=rs)
        skf.get_n_splits(x, y)
        print(y)
        for train_index, test_index in skf.split(np.zeros(x.shape[0]), y):
            print(train_index)
            x_train, x_test = x.iloc[train_index,:], x.iloc[test_index,:]
            y_train, y_test = y[train_index], y[test_index]

            #
            empty_train_columns =  []
            for col in x_train.columns.values:
                # all the values for this feature are null
                if sum(x_train[col].isnull()) == x_train.shape[0]:
                    empty_train_columns.append(col)
            imputed_columns=[c for c in x_train.columns if c not in empty_train_columns]
    #         x_train = x_train.dropna(axis=1,how='all').values

            #impute missing
            imp_knn = KNNImputer(missing_values=np.nan)

            imp_knn_x_train=pd.DataFrame(imp_knn.fit_transform(x_train),index=x_train.index,columns=x_train.columns)
            imp_knn_x_test=pd.DataFrame(imp_knn.transform(x_test),index=x_test.index,columns=x_test.columns)

            imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

            imp_mean_x_train=pd.DataFrame(imp_mean.fit_transform(x_train),index=x_train.index,columns=x_train.columns)
            imp_mean_x_test=pd.DataFrame(imp_mean.transform(x_test),index=x_test.index,columns=x_test.columns)

    #         print("Imputated training data - iron lvl",imp_knn_x_train['LabAdm_Iron'])
    #         print(x_train['LabAdm_Iron'])
    #         print(imp_mean_x_train.shape)

            for col in miss_knn_columns:
                x_train[col]=imp_knn_x_train[col]
                x_test[col]=imp_knn_x_test[col]
                if pd.isnull(x_train[col]).sum() > 0:
                    print(col,x_train.shape)
                    print('Null values for col, ',col,' are',pd.isnull(x_train[col]).sum())
                    print(x_train[col])
            for col in miss_mean_columns:
                x_train[col]=imp_mean_x_train[col]
                x_test[col]=imp_mean_x_test[col]
    #         print(x_train.shape)
    #         print(pd.isnull(x_train).sum())
    #         print(pd.isnull(imp_mean_x_train).sum())

        #smote
            over = SMOTE()
            under = RandomUnderSampler()
            steps = [('o', over),('u',under)]
            pipeline = Pipeline(steps=steps)
#             print('x_train', x_train.shape)
            x_train, y_train = pipeline.fit_resample(x_train, y_train)

        #model fit and prediction
        #create model
            model = clone(m)
        #fit

            clf = GridSearchCV(model, parameters,cv=5, n_jobs=None,scoring='r2')
            clf.fit(x_train, y_train)

            model = clf.best_estimator_
    #         #shap feature importance
    #         explainer = shap.KernelExplainer(model.predict,x_train)
    #         shap_test = explainer.shap_values(x_test,n_samples=1000)
    #         print(f"Shap values length: {len(shap_test)}\n")
    #         print(f"Sample shap value:\n{shap_test[0]}")
    #         shap_df = pd.DataFrame(shap_test.values, 
    #                        columns=shap_test.feature_names, 
    #                        index=x_test.index)
    #         print(np.isclose(model.predict(x_test), explainer.expected_value[0] + shap_df.sum(axis=1)))

    #         columns = shap_df.apply(np.abs).mean()\
    #                          .sort_values(ascending=False).index
    #         fig, ax = plt.subplots(1, 2, figsize=(11,4))
    #         sns.barplot(data=shap_df[columns].apply(np.abs), orient='h', 
    #                     ax=ax[0])
    #         ax[0].set_title("Mean absolute shap value")
    #         sns.boxplot(data=shap_df[columns], orient='h', ax=ax[1])
    #         ax[1].set_title("Distribution of shap values");
    #         plt.show()

            result = permutation_importance(model, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
            feature_importances = pd.DataFrame(result.importances_mean, index =x_test.columns,  columns=['importance']).sort_values('importance', ascending=False)

            print(feature_importances.head(20))
            fis.append(feature_importances)
    #     check common entries between x_train and x_test
    #         crs=[]
    #         print('common rows')
    #         for row in x_test:
    # #             print(row)
    #             cr = np.where((x_train == row).all(axis=1))
    #             if len(cr)>0:
    #                 crs.append(cr)
    # #             print(cr[0])
    # #             print(x_train[cr[0]])
    #         print(crs)

    #     #check duplicates for x_train
    #         num_dups=(x_train[:, np.newaxis] == x_train).all(axis=2).sum(axis=1)
    #         print('Duplicates ' ,np.where((num_dups>1)))

        #predict
            y_pred = clf.predict(x_test)
            out=(y_test,y_pred)
            outs.append(out)
            print('Best params')
            print(clf.best_estimator_.get_params())
    #         print("Accuracy: ",(np.trace(matrix))/matrix.sum()*100)
    df_concat = pd.concat(fis)
    print(df_concat.groupby(level=0).mean().sort_values('importance', ascending=False))
    outs.append(df_concat.groupby(level=0).mean().sort_values('importance', ascending=False))
    return outs

In [10]:
def regression_roc_auc_score(y_true, y_pred, num_rounds = 'exact'):
#   """
#   Computes Regression-ROC-AUC-score.
  
#   Parameters:
#   ----------
#   y_true: array-like of shape (n_samples,). Binary or continuous target variable.
#   y_pred: array-like of shape (n_samples,). Target scores.
#   num_rounds: int or string. If integer, number of random pairs of observations. 
#               If string, 'exact', all possible pairs of observations will be evaluated.
  
#   Returns:
#   -------
#   rroc: float. Regression-ROC-AUC-score.
#   """
  
#   import numpy as np
    
#   y_true = np.array(y_true)
#   y_pred = np.array(y_pred)

    num_pairs = 0
    num_same_sign = 0
#     print(y_true)
  
#     print('yield pairs')
#     for x in _yield_pairs(y_true, num_rounds):
#         print (x)
    for i, j in _yield_pairs(y_true, num_rounds):
#         print('i',i)
#         print('j',j)
#         print('y_true i',y_true[i])
#         print('y_true j',y_true[j])

        diff_true = y_true[i] - y_true[j]
        diff_score = y_pred[i] - y_pred[j]
        if diff_true * diff_score > 0:
          num_same_sign += 1
        elif diff_score == 0:
          num_same_sign += .5
        num_pairs += 1

    return num_same_sign / num_pairs


def _yield_pairs(y_true, num_rounds):
    """
    Returns pairs of valid indices. Indices must belong to observations having different values.

    Parameters:
    ----------
    y_true: array-like of shape (n_samples,). Binary or continuous target variable.
    num_rounds: int or string. If integer, number of random pairs of observations to return. 
              If string, 'exact', all possible pairs of observations will be returned.

    Yields:
    -------
    i, j: tuple of int of shape (2,). Indices referred to a pair of samples.

    """
#   import numpy as np
    if num_rounds == 'exact':
        for i in range(len(y_true)):
            for j in np.where((y_true != y_true[i]) & (np.arange(len(y_true)) > i))[0]:
                yield i, j     
    else:
        for r in range(num_rounds):
            i = np.random.choice(range(len(y_true)))
            j = np.random.choice(np.where(y_true != y_true[i])[0])
            yield i, j
            
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
def output_metrics(outs):
#     average_matrix=np.mean(matrices,axis=0)
    r2s = []
    maes=[]
    rmses=[]
    aucs=[]
    for out in outs[:-1]:
        r2 = r2_score(out[0],out[1])
        r2s.append(r2)
        mae = mean_absolute_error(out[0],out[1])
        maes.append(mae)
        mse = mean_squared_error(out[0],out[1])
        rmses.append(np.sqrt(mse))
        auc = regression_roc_auc_score(out[0],out[1])
        aucs.append(auc)
    r2=np.mean(r2s)
    mae=np.mean(maes)
    rmse=np.mean(rmses)
    auc=np.mean(aucs)
    print("r2 :",r2)
    print('mae:',mae)
    print('rmse:',rmse)
    print('auc:',auc)
    print('Feature Importance',outs[-1].head(10))
    return([outs,r2s,maes,rmses,aucs])

In [11]:
res={}

### Linear Regression

In [12]:
from sklearn.linear_model import LinearRegression
parameters = {'fit_intercept':[True,False]}
# for d in days_to_check:
outs=model_train(LinearRegression(),parameters)
res['OLS']=output_metrics(outs)
with open('results/'+'LR_hosp_final.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(res, file)

(1263, 356)
853
[5. 3. 2. ... 5. 1. 1.]
[   0    1    2 ... 1259 1260 1261]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                            importance
pre_internal_LabAdm_MCV_Hours               991.471335
pre_internal_LabDisch_Na_Hours              850.507256
pre_internal_LabDisch_K_Hours               815.298019
pre_internal_LabAdm_WBC_Hours               354.462432
pre_internal_LabAdm_Hb_Hours                354.462432
pre_internal_LabAdm_Platelets_Hours          33.441466
pre_internal_Lab_WBC_Count                   14.642991
pre_internal_Lab_MCV_Count                   13.544895
pre_internal_Lab_Na_Count                     2.970972
pre_internal_LOS                              2.334913
pre_internal_Lab_Cr_Count                     2.272507
pre_internal_LabAdm_Iron_AbnormalityCode      2.248313
pre_internal_MedDisch_ARB_Dose                2.058828
pre_internal_LabDisch_Iron_AbnormalityCode    1.739861
pre_internal_Valsartan Dose                   1.558586
pre_internal_ETL_ECG                          1.427959
pre_internal_Lab_EGFR_Count                   1.393317
pre_intern

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                                 importance
pre_internal_LabDisch_Na_Hours                  2378.759837
pre_internal_LabDisch_K_Hours                   2376.360214
pre_internal_Lab_Hb_Count                        481.633954
pre_internal_Lab_WBC_Count                       317.260106
pre_internal_Lab_MCV_Count                        10.824063
pre_internal_Lab_Na_Count                          2.336721
pre_internal_LabAdm_MCV_Hours                      2.056242
pre_internal_Lab_Platelets_Count                   1.901980
pre_internal_Lab_AST_Count                         1.217475
pre_internal_Lab_K_Count                           1.129827
pre_internal_Lab_ALT_Count                         1.004563
pre_internal_Lab_Cr_Count                          0.776891
pre_internal_LabDisch_Ferritin_AbnormalityCode     0.722895
pre_internal_LabAdm_Iron_AbnormalityCode           0.572626
pre_internal_LabAdm_Hb_Hours                       0.482082
pre_internal_LabAdm_WBC_Hours           

...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                                 importance
pre_internal_LabDisch_K_Hours                   1585.477076
pre_internal_LabDisch_Na_Hours                  1582.369780
pre_internal_LabAdm_HbA1C                          7.772585
pre_internal_LabDisch_Ferritin_AbnormalityCode     7.742515
pre_internal_LabDisch_HbA1C                        6.736380
pre_internal_LabAdm_Ferritin_AbnormalityCode       6.056807
pre_internal_Lab_MCV_Count                         5.653692
pre_internal_Lab_WBC_Count                         3.681694
pre_internal_Lab_Na_Count                          3.169524
pre_internal_Lab_K_Count                           1.846256
pre_internal_LabAdm_MCV_Hours                      1.571745
pre_internal_LabDisch_Hb_Hours                     1.190676
pre_internal_LabAdm_UricAcid                       1.089332
pre_internal_Furosemide Dose                       1.060548
pre_internal_LabAdm_Iron_AbnormalityCode           0.995213
pre_internal_Lab_EGFR_Count             

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                                importance
pre_internal_LabDisch_Na_Hours                  437.473283
pre_internal_LabDisch_K_Hours                   433.634946
pre_internal_Lab_Hb_Count                       127.640133
pre_internal_Lab_WBC_Count                       69.480695
pre_internal_Lab_MCV_Count                        6.460987
pre_internal_Lab_Na_Count                         3.281954
pre_internal_Lab_K_Count                          2.910691
pre_internal_LabAdm_Iron_AbnormalityCode          2.162711
pre_internal_LabDisch_Hb_Hours                    1.932657
pre_internal_MedDisch_ARB_Dose                    1.465397
pre_internal_LabDisch_WBC_Hours                   1.340347
pre_internal_LabDisch_Iron_AbnormalityCode        1.158258
pre_internal_Valsartan Dose                       1.077886
pre_internal_LOS                                  0.603513
pre_internal_LabDisch_Ferritin_AbnormalityCode    0.447062
pre_internal_Furosemide Dose                      0.4111

r2 : -14.309038371523267
mae: 1.6139289481581993
rmse: 3.5723698666070036
auc: 0.5608170375018678
Feature Importance                                       importance
pre_internal_LabDisch_Na_Hours       1386.047089
pre_internal_LabDisch_K_Hours        1372.237606
pre_internal_Lab_Hb_Count             172.444439
pre_internal_Lab_WBC_Count            157.741082
pre_internal_LabAdm_MCV_Hours          70.396312
pre_internal_LabAdm_Hb_Hours           34.470411
pre_internal_LabAdm_WBC_Hours          34.470411
pre_internal_LabAdm_Platelets_Hours    13.019405
pre_internal_Lab_MCV_Count              7.838917
pre_internal_MedDisch_ARB_Dose          4.941542


### Ridge Regression

In [13]:
from sklearn.linear_model import Ridge
parameters = {'solver':['lsqr', 'sag'],'alpha': [1e-3, 1e-2, 1e-1, 1, 10, 100],'fit_intercept':[True, False]}
# for d in days_to_check:
outs=model_train(Ridge(),parameters)
res['Ridge']=output_metrics(outs)
with open('results/'+'Ridge_hosp_final.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(res, file)

(1263, 356)
853
[5. 3. 2. ... 5. 1. 1.]
[   0    1    2 ... 1259 1260 1261]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                       importance
pre_internal_LabDisch_Bilirubin_Hours    0.243345
pre_internal_LabAdm_NT_proBNP_Hours      0.193083
pre_internal_VisitAnonId                 0.153654
pre_internal_LabAdm_INR_Hours            0.147519
pre_internal_LabDisch_AST_Hours          0.127553
pre_internal_LabAdm_UricAcid_Hours       0.078720
pre_internal_LabAdm_GGT_Hours            0.064536
pre_internal_LabDisch_GGT                0.058962
pre_internal_LabDisch_ALP_Hours          0.055714
pre_internal_LabAdm_Ur_Hours             0.051524
pre_internal_Troponin                    0.042895
pre_internal_LabAdm_Iron_Hours           0.038070
pre_internal_Age                         0.035601
pre_internal_LabAdm_Platelets            0.035466
pre_internal_LabDisch_Iron_Hours         0.024242
pre_internal_LabAdm_UricAcid             0.019789
pre_internal_LabAdm_Ferritin             0.015684
pre_internal_LabDisch_ALT_Hours          0.015105
pre_internal_LabAdm_Ferritin_Hours       0.014747


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                       importance
pre_internal_LabDisch_Bilirubin_Hours    0.216380
pre_internal_LabDisch_AST_Hours          0.187230
pre_internal_VisitAnonId                 0.182158
pre_internal_LabDisch_ALT_Hours          0.066059
pre_internal_LabAdm_Cr                   0.063849
pre_internal_LabDisch_ALT                0.037253
pre_internal_LabDisch_Cr                 0.032856
pre_internal_Age                         0.032390
pre_internal_LabDisch_Ferritin_Hours     0.020651
pre_internal_LabAdm_AST_Hours            0.018414
pre_internal_LabAdm_Platelets            0.017260
pre_internal_Exam_DBP                    0.016506
pre_internal_LabAdm_HbA1C_Hours          0.015564
pre_internal_LabDisch_Platelets_Hours    0.010572
pre_internal_LabAdm_Hb                   0.010183
pre_internal_BNP                         0.010144
pre_internal_ECG_HR                      0.009700
pre_internal_LabAdm_Ferritin_Hours       0.008801
pre_internal_FiscalPeriod                0.008414


...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                       importance
pre_internal_LabDisch_AST_Hours          0.204016
pre_internal_VisitAnonId                 0.163028
pre_internal_LabDisch_Bilirubin_Hours    0.095772
pre_internal_LabDisch_ALT_Hours          0.083963
pre_internal_BNP                         0.059123
pre_internal_Age                         0.051172
pre_internal_LabDisch_MCV_Hours          0.025458
pre_internal_LabAdm_HbA1C_Hours          0.024008
pre_internal_LabAdm_Platelets            0.020043
pre_internal_LabAdm_AST_Hours            0.019918
pre_internal_LabDisch_WBC_Hours          0.019600
pre_internal_LabAdm_BNP_Hours            0.019429
pre_internal_LabAdm_Bilirubin            0.017440
pre_internal_Exam_DBP                    0.013496
pre_internal_LabDisch_UricAcid           0.012015
pre_internal_LabAdm_ALT                  0.011938
pre_internal_LabAdm_UricAcid             0.011397
pre_internal_AdmissionDate               0.009223
pre_internal_LabDisch_Na_Hours           0.008070


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                       importance
pre_internal_VisitAnonId                 0.218959
pre_internal_LabDisch_Bilirubin_Hours    0.152086
pre_internal_LabAdm_Cr                   0.132260
pre_internal_LabAdm_ALT                  0.071859
pre_internal_LabDisch_AST_Hours          0.051346
pre_internal_LabDisch_GGT_Hours          0.041817
pre_internal_LabDisch_Cr                 0.038522
pre_internal_BNP                         0.034485
pre_internal_Age                         0.029763
pre_internal_LabAdm_Platelets            0.023609
pre_CSBC_pre_diag_date                   0.022309
pre_internal_ECG_HR                      0.018445
pre_internal_LabDisch_MCV_Hours          0.018304
pre_internal_LabAdm_AST_Hours            0.012068
pre_internal_LabDisch_Bilirubin          0.011977
pre_internal_MedDisch_ARB_Dose           0.011074
pre_internal_LabAdm_Bilirubin_Hours      0.010719
pre_internal_LabAdm_NT_proBNP_Hours      0.009853
pre_internal_LabAdm_Bilirubin            0.009844


r2 : -0.1050305319304854
mae: 1.3032751333487267
rmse: 1.556504760633948
auc: 0.60922380748534
Feature Importance                                        importance
pre_internal_VisitAnonId                 0.189115
pre_internal_LabDisch_AST_Hours          0.116411
pre_internal_LabAdm_Ur_Hours             0.082805
pre_internal_LabDisch_Bilirubin_Hours    0.059929
pre_internal_LabDisch_ALT_Hours          0.051292
pre_internal_LabDisch_Ferritin_Hours     0.043569
pre_internal_Age                         0.031577
pre_internal_LabAdm_HbA1C_Hours          0.030781
pre_internal_LabAdm_Platelets            0.021427
pre_internal_BNP                         0.015858


### Extra Trees Regression

In [14]:
from sklearn import ensemble
clf_et=ensemble.ExtraTreesRegressor()
parameters = {'n_estimators':[10,20,100]}
# for d in days_to_check:
outs=model_train(clf_et,parameters)
res['ETR']=output_metrics(outs)
with open('results/'+'ETR_hosp_final.pkl', 'wb') as file:
    # A new file will be created
    pickle.dump(res, file)

(1263, 356)
853
[5. 3. 2. ... 5. 1. 1.]
[   0    1    2 ... 1259 1260 1261]


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                                 importance
pre_internal_VisitAnonId                           0.048162
pre_internal_MrCareProviderAnonId                  0.019828
pre_CSBC_pre_diag_flag                             0.019405
pre_internal_MR_Unit                               0.017300
pre_internal_LabAdm_BNP_AbnormalityCode            0.014676
pre_internal_LabAdm_Bilirubin_AbnormalityCode      0.012344
pre_internal_BNP                                   0.011386
pre_internal_Metoprolol Dose                       0.010575
pre_internal_MedDisch_MRA_NoReason                 0.010018
pre_internal_hx_AlcoholExcess                      0.009907
pre_internal_LabAdm_Alb_AbnormalityCode            0.008772
pre_CSBC_pre_CRT_flag                              0.008531
pre_internal_Defibrillator                         0.008397
pre_internal_LabDisch_ALT_AbnormalityCode          0.008312
pre_internal_Exam_Weight                           0.007888
pre_CSBC_pre_valve_flag                 

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                                 importance
pre_internal_FiscalYear                            0.109412
pre_internal_Age                                   0.030522
pre_internal_hx_CKD                                0.027623
pre_internal_MedDisch_LoopDiureticDose             0.026660
pre_internal_Imaging_LVEF_Year                     0.022951
pre_internal_LabDisch_INR_AbnormalityCode          0.015190
pre_internal_LabDisch_Platelets_AbnormalityCode    0.015181
pre_internal_LabAdm_Platelets                      0.014806
pre_internal_hx_VTE                                0.014192
pre_internal_LabAdm_INR_AbnormalityCode            0.012041
pre_internal_LabAdm_BNP_AbnormalityCode            0.010842
pre_CSBC_pre_ICD_flag                              0.010748
pre_internal_MedDisch_ASA                          0.010538
pre_internal_DischargeMedRec                       0.010296
pre_internal_MedDisch_ARB_NoReason                 0.009554
pre_internal_NYHA                       

...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                           importance
pre_internal_FiscalYear                      0.126216
pre_internal_Age                             0.025207
pre_internal_MedDisch_Hydralazine            0.022539
pre_internal_DefibrillatorNoReason           0.016371
pre_internal_MedDisch_LoopDiureticDose       0.016281
pre_internal_hx_HF_Hospitalization           0.014157
pre_internal_VisitAnonId                     0.011826
pre_internal_Imaging_MR                      0.011691
pre_internal_LabDisch_GGT_AbnormalityCode    0.011567
pre_internal_MedDisch_MRA_NoReason           0.011326
pre_internal_LabDisch_INR_Hours              0.010282
pre_internal_LabDisch_AST_AbnormalityCode    0.010282
pre_internal_MedDisch_BB_Name                0.010136
pre_internal_LabAdm_Platelets                0.009069
pre_internal_Defibrillator                   0.009009
pre_internal_MRDx                            0.008885
pre_internal_MedDisch_NOAC                   0.008874
pre_internal_LabDisch_EGFR  

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


                                               importance
pre_internal_FiscalYear                          0.207340
pre_internal_Imaging_LVEF_Year                   0.022390
pre_internal_hx_DM                               0.022286
pre_internal_Exam_Weight                         0.021618
pre_internal_hx_AF                               0.021430
pre_internal_Age                                 0.015839
pre_internal_MedDisch_LoopDiureticDose           0.014819
pre_internal_LabAdm_TroponinI_AbnormalityCode    0.012515
pre_internal_hx_AlcoholExcess                    0.010514
pre_internal_hx_Malignancy                       0.010378
pre_internal_DischargeUnitCode                   0.010179
pre_CSBC_pre_diag_date                           0.009771
pre_internal_Lab_TroponinT_Count                 0.009604
pre_internal_Orthopnea                           0.009326
pre_internal_LabDisch_EGFR                       0.008942
pre_internal_NYHA                                0.008664
pre_CSBC_pre_d

r2 : -0.00045055541515221415
mae: 1.2640888293154882
rmse: 1.4839921724655365
auc: 0.5932492263868114
Feature Importance                                    importance
pre_internal_FiscalYear              0.071400
pre_internal_Age                     0.010882
pre_internal_hx_DM                   0.005534
pre_internal_VisitAnonId             0.004625
pre_CSBC_pre_ICD_flag                0.004387
pre_internal_MrCareProviderAnonId    0.004089
pre_internal_Exam_Weight             0.004077
pre_internal_MRDx                    0.003174
pre_internal_DischargeMedRec         0.002942
pre_internal_DischargeRxGiven        0.002693


In [15]:
df.dtypes

Unnamed: 0                                                int64
pre_internal_VisitAnonId                                  int64
pre_internal_PatientAnonId                                int64
pre_internal_MrCareProviderAnonId                         int64
pre_internal_MrCareProviderService                       object
pre_internal_Gender                                       int64
pre_internal_Age                                          int64
pre_internal_HealthAuthorityCode                         object
pre_internal_FacilityCode                                object
pre_internal_AdmissionDate                               object
pre_internal_DischargeDate                               object
pre_internal_MRDx                                        object
pre_internal_MR_Unit                                     object
pre_internal_FiscalPeriod                                object
pre_internal_FiscalYear                                   int64
pre_internal_LOS                        